In [1]:
##This Iteration calculates the YoY instead of QoQ revenue change as the y_tag
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
import scipy.sparse
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary


# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

/home/felixtsui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)



access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

# #Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [3]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [ ]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
#             if count == 500: break  # initially limit for time sake
    

In [ ]:
#Read from the pre prepared vocab file
allword=set()
with open('vocab1.csv', 'r') as f:
    for line in f:
        allword.add(line.strip())

In [ ]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None) 
print("Vocabulary size: {:,}".format(vocab.size))

In [ ]:
#Crate the padding
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [ ]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [ ]:
#Set of all the stocks of interest
trainstock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']


In [ ]:
##Converting the string of data into dictionary


for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in trainstock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
                ##Storing in content of each tweet and length of each tweet in sub level variable
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    ##Storing in content of each tweet and length of each tweet in the top level variable
    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [ ]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

In [ ]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_revQ1=0
    last_revQ2=0
    last_revQ3=0
    last_revQ4=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2>3:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Calculate change of revenue into the y_label 
                            if j["field"] =="totalrevenue":
                                if count2%4==0: 
                                    revenue_cal =round(float(j['value'])/float(last_revQ1) - 1,3)
                                    last_revQ1=j['value']
                                    if revenue_cal>0.1:
                                        revenue_change.append(3)
                                    elif revenue_cal>0:
                                        revenue_change.append(2)
                                    elif revenue_cal>-0.1:
                                        revenue_change.append(1)
                                    else:
                                        revenue_change.append(0)
                                elif count2%4==1: 
                                    revenue_cal =round(float(j['value'])/float(last_revQ2) - 1,3)
                                    last_revQ2=j['value']
                                    if revenue_cal>0.1:
                                        revenue_change.append(3)
                                    elif revenue_cal>0:
                                        revenue_change.append(2)
                                    elif revenue_cal>-0.1:
                                        revenue_change.append(1)
                                    else:
                                        revenue_change.append(0)
                                elif count2%4==2: 
                                    revenue_cal =round(float(j['value'])/float(last_revQ3) - 1,3)
                                    last_revQ3=j['value']
                                    if revenue_cal>0.1:
                                        revenue_change.append(3)
                                    elif revenue_cal>0:
                                        revenue_change.append(2)
                                    elif revenue_cal>-0.1:
                                        revenue_change.append(1)
                                    else:
                                        revenue_change.append(0)
                                else: 
                                    revenue_cal =round(float(j['value'])/float(last_revQ4) - 1,3)
                                    last_revQ4=j['value']
                                    if revenue_cal>0.1:
                                        revenue_change.append(3)
                                    elif revenue_cal>0:
                                        revenue_change.append(2)
                                    elif revenue_cal>-0.1:
                                        revenue_change.append(1)
                                    else:
                                        revenue_change.append(0)
                        ##Storing each of the initial 4 quarters of revenue for calculation            
                        elif count2==1:
                            if j["field"] =="totalrevenue":      
                                last_revQ2=j['value']                            
                        elif count2==2:
                            if j["field"] =="totalrevenue":      
                                last_revQ3=j['value']                            
                        elif count2==3:
                            if j["field"] =="totalrevenue":      
                                last_revQ4=j['value']                            
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_revQ1=j['value']
                              
                    count2+=1

            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [ ]:
##Writing y_tag to file
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [ ]:
##Creating the train/Dev/test var
train_x_init = []
train_n_init = []
train_y_init = []
dev_x_init = []
dev_n_init = []
dev_y_init = []
test_x_init = []
test_n_init = []
test_y_init = []


In [ ]:
##Putting the earlier dictinoary into the train var in order of quarter
trainquarter=['16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

for i in trainstock:
    for j in range(len(trainquarter)):
        try:
                train_x_init.append(InputX [i][trainquarter[j]])
                train_n_init.append(InputN [i][trainquarter[j]])
                length=len(InputN [i][trainquarter[j]])           
                train_y_init.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass



In [ ]:
##Putting the earlier dictinoary into the dev/test var in order of quarter
teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']

testquarter=['18Q2','18Q3']

for i in teststock:
    for j in range(len(testquarter)):
      
        try:
            if testquarter[j]=='18Q3':
                test_x_init.append(InputX [i][testquarter[j]])
                test_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                test_y_init.append([Outputy [i]['revenuechange'][j]]*length)   
            
            elif testquarter[j]=='18Q2':
                dev_x_init.append(InputX [i][testquarter[j]])
                dev_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                dev_y_init.append([Outputy [i]['revenuechange'][j]]*length)     
               
      
        except:
            pass

In [ ]:
##Creating the refined train/Dev/test var by flattening the shape of the original var

train_x_init2 = []
train_n_init2 = []
train_y_init2 = []
dev_x_init2 = []
dev_n_init2 = []
dev_y_init2 = []
test_x_init2 = []
test_n_init2 = []
test_y_init2 = []

for i in range(len(train_x_init)):
    for j in range(len(train_x_init[i])):
     
        train_x_init2.append(train_x_init[i][j])
        train_n_init2.append(train_n_init[i][j])
        train_y_init2.append(train_y_init[i][j])
        
for i in range(len(dev_x_init)):
    for j in range(len(dev_x_init[i])):
        dev_x_init2.append(dev_x_init[i][j])
        dev_n_init2.append(dev_n_init[i][j])
        dev_y_init2.append(dev_y_init[i][j])
        
for i in range(len(test_x_init)):
    for j in range(len(test_x_init[i])):
        test_x_init2.append(test_x_init[i][j])
        test_n_init2.append(test_n_init[i][j])
        test_y_init2.append(test_y_init[i][j])



# NEURAL BAG OF WORDS MODEL #

In [ ]:
##Padding the refined var
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr 

In [ ]:
##Creating the final refined train/Dev/test var by padding it
train_x = pad_np_array2(train_x_init2, max_len = 10, pad_id=0)
train_n = np.array(train_n_init2)
train_y = np.array(train_y_init2)

test_x = pad_np_array2(test_x_init2, max_len = 10, pad_id=0)
test_n = np.array(test_n_init2)
test_y = np.array(test_y_init2)

dev_x = pad_np_array2(dev_x_init2, max_len = 10, pad_id=0)
dev_n = np.array(dev_n_init2)
dev_y = np.array(dev_y_init2)

In [ ]:
##Creat histogram to see the distribution of the testing data tag
lbl, cnts = np.unique(test_y, return_counts=True)
print(list(zip(lbl, cnts)))
print(lbl)
print(cnts)

plt.hist((cnts[0]*[lbl[0]],cnts[1]*[lbl[1]],cnts[2]*[lbl[2]]))

In [ ]:
##Creat histogram to see the distribution of the training data tag
lbl2, cnts2 = np.unique(train_y, return_counts=True)
print(list(zip(lbl2, cnts2)))
print(lbl2[0])
print(cnts2[0])


plt.hist((cnts2[0]*[lbl2[0]],cnts2[1]*[lbl2[1]],cnts[2]*[lbl[2]]))


In [ ]:
##Setting up the NBoW Model
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 125

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=4,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 125 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

In [ ]:
##Setting up the NBoW Model
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=4,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

In [ ]:
##Training the NBoW Model
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=125, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

In [ ]:
##Calculating the Accuracy

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

In [ ]:
##Calculating the Precision
from sklearn.metrics import average_precision_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]    
average_precision = average_precision_score(y_pred,test_y)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

In [ ]:
##Loss Analysis
counttest=[]

for i in range(len(test_y_init)):
    
    counttest.append(len(test_y_init[i]))

count=0
for i in range(len(counttest)):
 
    lbl, cnts = np.unique(test_y[count:count+counttest[i]], return_counts=True)
    lbl2, cnts2 = np.unique (y_pred[count:count+counttest[i]], return_counts=True)
    print(teststock[i])
    print(list(zip(lbl2, cnts2)))
    print(lbl[0], "vs", round(sum(lbl2*cnts2)/sum(cnts2),0))
    count+=counttest[i]

    

In [ ]:
##Histogram of the prediction
lbl, cnts = np.unique(y_pred, return_counts=True)
print(list(zip(lbl, cnts)))
print(lbl)
print(cnts)


plt.hist((cnts[0]*[lbl[0]],cnts[1]*[lbl[1]],cnts[2]*[lbl[2]]))